In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

###device config

In [2]:
# check for CUDA availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
# load the breast cancer dataset
data = load_breast_cancer()
X = data.data
y = data.target

In [4]:
# split the dataset into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
print(X.shape)
print(X_train.shape) #xtrain sahpe
print(X_test.shape)

(569, 30)
(455, 30)
(114, 30)


In [6]:
# standardize the data using Standard sclaer
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [7]:
# convert data numpy to PyTorch tensors and move it to GPU
X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.float32).to(device)
X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test = torch.tensor(y_test, dtype=torch.float32).to(device)

In [8]:
# define the neural network architecture

class NeuralNet(nn.Module): #import torch.nn

  def __init__(self, input_size, hidden_size, output_size): #feature = input #hidden how many nuron should #output neoron class ekta samanai
    super(NeuralNet, self).__init__() #checking parameters validation. super is used to call the parent class constructor
    self.fc1 = nn.Linear(input_size, hidden_size) #fc1 fully connected layer hidden layers me
    self.relu = nn.ReLU() #non linear
    self.fc2 = nn.Linear(hidden_size, output_size)  #layer hidden layers me
    self.sigmoid = nn.Sigmoid() #output

  def forward(self, x): #forward propogation
    out = self.fc1(x) #input
    out = self.relu(out)
    out = self.fc2(out)
    out = self.sigmoid(out)
    return out

In [16]:
# define hyperparameters #backward prop
input_size = X_train.shape[1] #features gana
hidden_size = 128 #nuron gana 128 32
output_size = 1 #sigmoid nisa
learning_rate = 0.001
num_epochs = 100 #epoch

In [17]:
# initialize the neural network and move it the GPU
model = NeuralNet(input_size, hidden_size, output_size).to(device)

In [18]:
# define loss and the optimizer
criterion = nn.BCELoss() #binary cross entropy
optimizer = optim.Adam(model.parameters(), lr=learning_rate) #uda optim adam optimizer

Training the model

In [19]:
# training the model
for epoch in range(num_epochs): #epochs
  model.train()
  optimizer.zero_grad() #clear the gradients
  outputs = model(X_train) #generate predictions
  loss = criterion(outputs, y_train.view(-1,1))  #loss function
  loss.backward() #backpropagation
  optimizer.step() #update weights

  # claculate accuracy
  with torch.no_grad():
    predicted = outputs.round()
    correct = (predicted == y_train.view(-1,1)).float().sum() #calculate the number of correct predictions
    accuracy = correct/y_train.size(0)

  if (epoch+1) % 10 == 0:
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss : {loss.item():.4f}, Accuracy: {accuracy.item() * 100:.2f}%")

Epoch [10/100], Loss : 0.5049, Accuracy: 93.41%
Epoch [20/100], Loss : 0.3524, Accuracy: 93.85%
Epoch [30/100], Loss : 0.2574, Accuracy: 94.07%
Epoch [40/100], Loss : 0.1984, Accuracy: 95.16%
Epoch [50/100], Loss : 0.1614, Accuracy: 95.82%
Epoch [60/100], Loss : 0.1368, Accuracy: 96.70%
Epoch [70/100], Loss : 0.1194, Accuracy: 97.36%
Epoch [80/100], Loss : 0.1063, Accuracy: 97.58%
Epoch [90/100], Loss : 0.0962, Accuracy: 98.02%
Epoch [100/100], Loss : 0.0882, Accuracy: 98.24%


In [20]:
# evaluation on training set
model.eval()
with torch.no_grad():
  outputs = model(X_train)
  predicted = outputs.round()
  correct = (predicted == y_train.view(-1,1)).float().sum()
  accuracy = correct/y_train.size(0)
  print(f"Accuracy on training data: {accuracy.item() * 100:.2f}%")

Accuracy on training data: 98.24%


In [21]:
# evaluation on test set
model.eval()
with torch.no_grad():
  outputs = model(X_test)
  predicted = outputs.round()
  correct = (predicted == y_test.view(-1,1)).float().sum()
  accuracy = correct/y_test.size(0)
  print(f"Accuracy on test data: {accuracy.item() * 100:.2f}%")

Accuracy on test data: 98.25%
